<a href="https://colab.research.google.com/github/hannahbanjo/AssociationOfDataScience/blob/main/Datathon_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Datathon Workshop: Complete Guide
### Data Cleaning • EDA • Business Questions • Stakeholder Presentations

**Workshop Goal:** Learn the essential skills to succeed in datathons by mastering data preparation, exploratory analysis, business problem-solving, and presentation techniques.

---

##Table of Contents
1. [Setup & Imports](#setup)
2. [Data Cleaning](#cleaning)
3. [Exploratory Data Analysis (EDA)](#eda)
4. [Answering Business Questions](#business)
5. [Creating Non-Technical Presentations](#presentations)
6. [Datathon Best Practices](#best-practices)

<a id='setup'></a>
## 1. Setup & Imports

First, let's import all the libraries we'll need for data manipulation, visualization, and analysis.

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Statistical analysis
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("All libraries imported successfully!")

### Generate Sample Dataset

For this workshop, we'll create a realistic e-commerce dataset with common data quality issues.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate sample e-commerce data
n_records = 1000

# Create base data
data = {
    'customer_id': range(1, n_records + 1),
    'age': np.random.randint(18, 75, n_records),
    'gender': np.random.choice(['Male', 'Female', 'Other'], n_records, p=[0.48, 0.48, 0.04]),
    'purchase_amount': np.random.gamma(2, 50, n_records),
    'items_purchased': np.random.poisson(3, n_records),
    'days_since_last_purchase': np.random.exponential(30, n_records),
    'customer_segment': np.random.choice(['Premium', 'Regular', 'New'], n_records, p=[0.15, 0.6, 0.25]),
    'marketing_channel': np.random.choice(['Email', 'Social Media', 'Direct', 'Referral'], n_records),
    'product_category': np.random.choice(['Electronics', 'Clothing', 'Home', 'Books', 'Sports'], n_records),
    'satisfaction_score': np.random.randint(1, 6, n_records),
    'discount_used': np.random.choice([True, False], n_records, p=[0.3, 0.7]),
}

df = pd.DataFrame(data)

# Introduce realistic data quality issues

# 1. Missing values (10-15% in different columns)
missing_indices_age = np.random.choice(df.index, size=int(0.12 * n_records), replace=False)
df.loc[missing_indices_age, 'age'] = np.nan

missing_indices_satisfaction = np.random.choice(df.index, size=int(0.08 * n_records), replace=False)
df.loc[missing_indices_satisfaction, 'satisfaction_score'] = np.nan

missing_indices_channel = np.random.choice(df.index, size=int(0.05 * n_records), replace=False)
df.loc[missing_indices_channel, 'marketing_channel'] = np.nan

# 2. Duplicates (3%)
duplicate_rows = df.sample(n=int(0.03 * n_records))
df = pd.concat([df, duplicate_rows], ignore_index=True)

# 3. Outliers in purchase_amount (add some extreme values)
outlier_indices = np.random.choice(df.index, size=20, replace=False)
df.loc[outlier_indices, 'purchase_amount'] = np.random.uniform(1000, 5000, 20)

# 4. Data type issues (convert some numeric to string)
df['items_purchased'] = df['items_purchased'].astype(str)

# 5. Inconsistent formatting in categorical data
gender_variations = {'Male': ['Male', 'male', 'M', 'MALE'],
                    'Female': ['Female', 'female', 'F', 'FEMALE'],
                    'Other': ['Other', 'other', 'O']}

for standard, variations in gender_variations.items():
    mask = df['gender'] == standard
    if mask.sum() > 0:
        sample_size = min(int(mask.sum() * 0.15), mask.sum())
        sample_indices = df[mask].sample(n=sample_size).index
        df.loc[sample_indices, 'gender'] = np.random.choice(variations, sample_size)

print(f"Dataset created with {len(df)} records (including duplicates)")
print(f"   Original records: {n_records}")
print(f"   Duplicate records added: {len(df) - n_records}")
print("\nData quality issues introduced:")
print(f"   - Missing values in age, satisfaction_score, marketing_channel")
print(f"   - {len(df) - n_records} duplicate rows")
print(f"   - Outliers in purchase_amount")
print(f"   - items_purchased stored as string instead of numeric")
print(f"   - Inconsistent gender formatting")

In [ ]:
# Preview the data
print("First 5 rows of the dataset:")
df.head()

<a id='cleaning'></a>
## 2. Data Cleaning

Data cleaning is the foundation of good analysis. **80% of data science work is cleaning data!**

### 2.1 Initial Data Inspection

In [ ]:
def inspect_data(df):
    """
    Comprehensive data inspection function
    """
    print("=" * 60)
    print("DATA INSPECTION REPORT")
    print("=" * 60)

    # Basic info
    print(f"\nDataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Data types
    print("\n📋 Data Types:")
    print(df.dtypes)

    # Missing values
    print("\nMissing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing[missing > 0],
        'Percentage': missing_pct[missing > 0]
    }).sort_values('Percentage', ascending=False)

    if len(missing_df) > 0:
        print(missing_df)
    else:
        print("No missing values found!")

    # Duplicates
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate Rows: {duplicates} ({duplicates/len(df)*100:.2f}%)")

    # Numeric columns summary
    print("\nNumeric Columns Summary:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(df[numeric_cols].describe())

    # Categorical columns
    print("\nCategorical Columns:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        unique_count = df[col].nunique()
        print(f"  {col}: {unique_count} unique values")
        if unique_count <= 10:
            print(f"    Values: {df[col].value_counts().to_dict()}")

    print("\n" + "=" * 60)

# Run inspection
inspect_data(df)

### 2.2 Handling Duplicates

In [ ]:
# Check for duplicates
print(f"Total duplicates before removal: {df.duplicated().sum()}")

# Remove duplicates
df_clean = df.drop_duplicates()

print(f"Total duplicates after removal: {df_clean.duplicated().sum()}")
print(f"Rows removed: {len(df) - len(df_clean)}")
print(f"\nDataset shape after removing duplicates: {df_clean.shape}")

### 2.3 Fixing Data Types

In [ ]:
# Convert items_purchased back to numeric
print("Before conversion:")
print(f"items_purchased dtype: {df_clean['items_purchased'].dtype}")

df_clean['items_purchased'] = pd.to_numeric(df_clean['items_purchased'], errors='coerce')

print("\nAfter conversion:")
print(f"items_purchased dtype: {df_clean['items_purchased'].dtype}")
print(f"\nData type corrected for items_purchased")

### 2.4 Standardizing Categorical Variables

In [ ]:
# Check current gender values
print("Gender values before standardization:")
print(df_clean['gender'].value_counts())

# Standardize gender column
gender_mapping = {
    'male': 'Male', 'M': 'Male', 'MALE': 'Male',
    'female': 'Female', 'F': 'Female', 'FEMALE': 'Female',
    'other': 'Other', 'O': 'Other'
}

df_clean['gender'] = df_clean['gender'].replace(gender_mapping)

print("\nGender values after standardization:")
print(df_clean['gender'].value_counts())
print(f"\nGender column standardized")

### 2.5 Handling Missing Values

In [ ]:
# Strategy 1: Fill numeric missing values with median
print("Missing values before imputation:")
print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])

# For age: use median
age_median = df_clean['age'].median()
df_clean['age'].fillna(age_median, inplace=True)

# For satisfaction_score: use mode (most common)
satisfaction_mode = df_clean['satisfaction_score'].mode()[0]
df_clean['satisfaction_score'].fillna(satisfaction_mode, inplace=True)

# For marketing_channel: create 'Unknown' category
df_clean['marketing_channel'].fillna('Unknown', inplace=True)

print("\nMissing values after imputation:")
print(df_clean.isnull().sum().sum())  # Total missing values
print(f"\nAll missing values handled")

### 2.6 Detecting and Handling Outliers

In [ ]:
def detect_outliers_iqr(df, column):
    """
    Detect outliers using IQR (Interquartile Range) method
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return outliers, lower_bound, upper_bound

# Detect outliers in purchase_amount
outliers, lower, upper = detect_outliers_iqr(df_clean, 'purchase_amount')

print(f"Outlier detection for 'purchase_amount':")
print(f"  Lower bound: ${lower:.2f}")
print(f"  Upper bound: ${upper:.2f}")
print(f"  Number of outliers: {len(outliers)} ({len(outliers)/len(df_clean)*100:.2f}%)")
print(f"\nOutlier range: ${outliers['purchase_amount'].min():.2f} - ${outliers['purchase_amount'].max():.2f}")

# Visualize outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Before handling outliers
axes[0].boxplot(df_clean['purchase_amount'])
axes[0].set_title('Purchase Amount - With Outliers')
axes[0].set_ylabel('Amount ($)')

# Cap outliers at upper bound
df_clean['purchase_amount_capped'] = df_clean['purchase_amount'].clip(upper=upper)

# After handling outliers
axes[1].boxplot(df_clean['purchase_amount_capped'])
axes[1].set_title('Purchase Amount - Outliers Capped')
axes[1].set_ylabel('Amount ($)')

plt.tight_layout()
plt.show()

print(f"\nOutliers capped at ${upper:.2f}")

### 2.7 Data Cleaning Summary

In [ ]:
print("=" * 60)
print("DATA CLEANING SUMMARY")
print("=" * 60)
print(f"\nDuplicates removed: {len(df) - len(df_clean)} rows")
print(f"Data types corrected: items_purchased")
print(f"Categorical standardization: gender column")
print(f"Missing values imputed:")
print(f"   - age: filled with median ({age_median:.0f})")
print(f"   - satisfaction_score: filled with mode ({satisfaction_mode})")
print(f"   - marketing_channel: filled with 'Unknown'")
print(f"Outliers handled: purchase_amount capped at ${upper:.2f}")
print(f"\nFinal dataset: {df_clean.shape[0]} rows × {df_clean.shape[1]} columns")
print("=" * 60)

<a id='eda'></a>
## 3. Exploratory Data Analysis (EDA)

EDA helps us understand patterns, relationships, and insights in the data.

### 3.1 Univariate Analysis

In [ ]:
# Distribution of purchase amounts
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df_clean['purchase_amount_capped'], bins=30, edgecolor='black', alpha=0.7, color='teal')
axes[0].set_xlabel('Purchase Amount ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Purchase Amounts')
axes[0].axvline(df_clean['purchase_amount_capped'].mean(), color='red', linestyle='--', label=f"Mean: ${df_clean['purchase_amount_capped'].mean():.2f}")
axes[0].axvline(df_clean['purchase_amount_capped'].median(), color='orange', linestyle='--', label=f"Median: ${df_clean['purchase_amount_capped'].median():.2f}")
axes[0].legend()

# Box plot
axes[1].boxplot(df_clean['purchase_amount_capped'], vert=True)
axes[1].set_ylabel('Purchase Amount ($)')
axes[1].set_title('Purchase Amount Box Plot')

plt.tight_layout()
plt.show()

# Summary statistics
print("Purchase Amount Statistics:")
print(df_clean['purchase_amount_capped'].describe())

In [ ]:
# Categorical variable analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Customer segment distribution
segment_counts = df_clean['customer_segment'].value_counts()
axes[0, 0].bar(segment_counts.index, segment_counts.values, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Customer Segment Distribution')
axes[0, 0].set_ylabel('Count')

# Marketing channel
channel_counts = df_clean['marketing_channel'].value_counts()
axes[0, 1].bar(channel_counts.index, channel_counts.values, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('Marketing Channel Distribution')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# Product category
category_counts = df_clean['product_category'].value_counts()
axes[1, 0].bar(category_counts.index, category_counts.values, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Product Category Distribution')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=45)

# Satisfaction score
satisfaction_counts = df_clean['satisfaction_score'].value_counts().sort_index()
axes[1, 1].bar(satisfaction_counts.index, satisfaction_counts.values, color='plum', edgecolor='black')
axes[1, 1].set_title('Satisfaction Score Distribution')
axes[1, 1].set_xlabel('Score')
axes[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.show()

### 3.2 Bivariate Analysis

In [ ]:
# Relationship between age and purchase amount
plt.figure(figsize=(12, 6))
plt.scatter(df_clean['age'], df_clean['purchase_amount_capped'], alpha=0.5, c='teal')
plt.xlabel('Age')
plt.ylabel('Purchase Amount ($)')
plt.title('Age vs Purchase Amount')

# Add trend line
z = np.polyfit(df_clean['age'].dropna(), df_clean.loc[df_clean['age'].notna(), 'purchase_amount_capped'], 1)
p = np.poly1d(z)
plt.plot(df_clean['age'].sort_values(), p(df_clean['age'].sort_values()), "r--", alpha=0.8, label='Trend line')
plt.legend()
plt.show()

# Calculate correlation
correlation = df_clean[['age', 'purchase_amount_capped']].corr().iloc[0, 1]
print(f"Correlation between age and purchase amount: {correlation:.3f}")

In [ ]:
# Purchase amount by customer segment
plt.figure(figsize=(12, 6))
df_clean.boxplot(column='purchase_amount_capped', by='customer_segment', ax=plt.gca())
plt.title('Purchase Amount by Customer Segment')
plt.suptitle('')  # Remove default title
plt.xlabel('Customer Segment')
plt.ylabel('Purchase Amount ($)')
plt.show()

# Summary statistics by segment
print("\nPurchase Amount by Customer Segment:")
print(df_clean.groupby('customer_segment')['purchase_amount_capped'].describe())

In [ ]:
# Discount usage vs purchase amount
fig, ax = plt.subplots(figsize=(10, 6))
discount_groups = df_clean.groupby('discount_used')['purchase_amount_capped'].mean()
colors = ['lightcoral' if not x else 'lightgreen' for x in discount_groups.index]
ax.bar(['No Discount', 'Discount Used'], discount_groups.values, color=colors, edgecolor='black')
ax.set_ylabel('Average Purchase Amount ($)')
ax.set_title('Average Purchase Amount: Discount vs No Discount')

# Add value labels on bars
for i, v in enumerate(discount_groups.values):
    ax.text(i, v + 2, f'${v:.2f}', ha='center', fontweight='bold')

plt.show()

print(f"Average purchase with discount: ${discount_groups[True]:.2f}")
print(f"Average purchase without discount: ${discount_groups[False]:.2f}")
print(f"Difference: ${discount_groups[True] - discount_groups[False]:.2f}")

### 3.3 Correlation Analysis

In [ ]:
# Correlation heatmap for numeric variables
numeric_cols = ['age', 'purchase_amount_capped', 'items_purchased', 'days_since_last_purchase', 'satisfaction_score']
correlation_matrix = df_clean[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Numeric Variables')
plt.tight_layout()
plt.show()

# Identify strong correlations
print("\nStrong Correlations (|r| > 0.3):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.3:
            print(f"{correlation_matrix.columns[i]} <-> {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

### 3.4 Advanced EDA - Interactive Visualizations

In [ ]:
# Interactive scatter plot with Plotly
fig = px.scatter(df_clean,
                 x='age',
                 y='purchase_amount_capped',
                 color='customer_segment',
                 size='items_purchased',
                 hover_data=['satisfaction_score', 'marketing_channel'],
                 title='Purchase Behavior Analysis',
                 labels={'purchase_amount_capped': 'Purchase Amount ($)',
                        'age': 'Age (years)'})
fig.show()

print("Insight: Hover over points to see detailed customer information")

In [ ]:
# Purchase by category and channel - Interactive
category_channel = df_clean.groupby(['product_category', 'marketing_channel']).size().reset_index(name='count')

fig = px.sunburst(category_channel,
                  path=['product_category', 'marketing_channel'],
                  values='count',
                  title='Product Categories and Marketing Channels')
fig.show()

print("💡 Insight: Click on segments to drill down into the data")

<a id='business'></a>
## 4. Answering Business Questions

Transform data insights into actionable business value.

### The Framework:
1. **Understand the Question** - What decision does this answer inform?
2. **Identify Relevant Data** - Which features/tables matter?
3. **Analyze & Quantify** - Use statistics, trends, comparisons
4. **Validate Results** - Check for biases and edge cases
5. **Craft the Answer** - Clear, specific, actionable

### Business Question 1: Which customer segment has the highest revenue potential?

In [ ]:
# Calculate total and average revenue by segment
segment_analysis = df_clean.groupby('customer_segment').agg({
    'purchase_amount_capped': ['sum', 'mean', 'count'],
    'items_purchased': 'mean',
    'satisfaction_score': 'mean'
}).round(2)

segment_analysis.columns = ['Total Revenue', 'Avg Purchase', 'Customer Count', 'Avg Items', 'Avg Satisfaction']
segment_analysis = segment_analysis.sort_values('Total Revenue', ascending=False)

print("Customer Segment Analysis:")
print(segment_analysis)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total revenue
axes[0].bar(segment_analysis.index, segment_analysis['Total Revenue'],
            color=['gold', 'silver', '#CD7F32'], edgecolor='black')
axes[0].set_title('Total Revenue by Customer Segment')
axes[0].set_ylabel('Total Revenue ($)')
axes[0].tick_params(axis='x', rotation=0)

# Average purchase
axes[1].bar(segment_analysis.index, segment_analysis['Avg Purchase'],
            color=['teal', 'skyblue', 'lightblue'], edgecolor='black')
axes[1].set_title('Average Purchase by Customer Segment')
axes[1].set_ylabel('Average Purchase ($)')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

print("\nKEY INSIGHT:")
top_segment = segment_analysis.index[0]
print(f"{top_segment} customers generate the highest total revenue")
print(f"Average purchase: ${segment_analysis.loc[top_segment, 'Avg Purchase']:.2f}")
print(f"Customer count: {segment_analysis.loc[top_segment, 'Customer Count']:.0f}")
print(f"\nRECOMMENDATION: Prioritize retention programs for {top_segment} customers")

### Business Question 2: Which marketing channel is most effective?

In [ ]:
# Analyze effectiveness by channel
channel_analysis = df_clean.groupby('marketing_channel').agg({
    'purchase_amount_capped': ['mean', 'sum'],
    'customer_id': 'count',
    'satisfaction_score': 'mean',
    'items_purchased': 'mean'
}).round(2)

channel_analysis.columns = ['Avg Purchase', 'Total Revenue', 'Customers', 'Avg Satisfaction', 'Avg Items']

# Calculate conversion value (avg purchase * customers)
channel_analysis['Total Value'] = channel_analysis['Avg Purchase'] * channel_analysis['Customers']
channel_analysis = channel_analysis.sort_values('Avg Purchase', ascending=False)

print("Marketing Channel Effectiveness:")
print(channel_analysis)

# Visualize
fig = go.Figure()

fig.add_trace(go.Bar(
    x=channel_analysis.index,
    y=channel_analysis['Avg Purchase'],
    name='Avg Purchase',
    marker_color='lightblue'
))

fig.add_trace(go.Scatter(
    x=channel_analysis.index,
    y=channel_analysis['Avg Satisfaction'],
    name='Avg Satisfaction',
    yaxis='y2',
    marker_color='red',
    mode='lines+markers'
))

fig.update_layout(
    title='Marketing Channel Performance',
    xaxis_title='Marketing Channel',
    yaxis_title='Average Purchase ($)',
    yaxis2=dict(title='Average Satisfaction', overlaying='y', side='right', range=[0, 5]),
    hovermode='x unified'
)

fig.show()

print("\nKEY INSIGHTS:")
best_channel = channel_analysis.index[0]
print(f"{best_channel} has the highest average purchase: ${channel_analysis.loc[best_channel, 'Avg Purchase']:.2f}")
print(f"Customer satisfaction via {best_channel}: {channel_analysis.loc[best_channel, 'Avg Satisfaction']:.2f}/5")
print(f"\nRECOMMENDATION: Increase budget allocation to {best_channel} marketing")

### Business Question 3: What drives customer satisfaction?

In [ ]:
# Analyze factors affecting satisfaction
from scipy.stats import pearsonr

# Satisfaction by purchase amount groups
df_clean['purchase_group'] = pd.cut(df_clean['purchase_amount_capped'],
                                     bins=[0, 50, 100, 200, 1000],
                                     labels=['Low', 'Medium', 'High', 'Very High'])

satisfaction_by_purchase = df_clean.groupby('purchase_group')['satisfaction_score'].mean().sort_index()

plt.figure(figsize=(10, 6))
bars = plt.bar(satisfaction_by_purchase.index, satisfaction_by_purchase.values,
               color=['lightcoral', 'lightyellow', 'lightgreen', 'darkgreen'], edgecolor='black')
plt.xlabel('Purchase Amount Group')
plt.ylabel('Average Satisfaction Score')
plt.title('Customer Satisfaction by Purchase Amount')
plt.ylim(0, 5)
plt.axhline(y=df_clean['satisfaction_score'].mean(), color='red', linestyle='--',
            label=f"Overall Avg: {df_clean['satisfaction_score'].mean():.2f}")

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}', ha='center', va='bottom', fontweight='bold')

plt.legend()
plt.show()

# Satisfaction by segment and discount usage
satisfaction_factors = df_clean.groupby(['customer_segment', 'discount_used'])['satisfaction_score'].mean().unstack()

satisfaction_factors.plot(kind='bar', figsize=(10, 6), color=['lightcoral', 'lightgreen'], edgecolor='black')
plt.title('Satisfaction Score: Segment & Discount Impact')
plt.xlabel('Customer Segment')
plt.ylabel('Average Satisfaction Score')
plt.legend(['No Discount', 'Discount Used'])
plt.xticks(rotation=0)
plt.ylim(0, 5)
plt.tight_layout()
plt.show()

print("\nKEY INSIGHTS:")
print(f"Higher purchase amounts correlate with higher satisfaction")
print(f"Discount usage improves satisfaction across all segments")
print(f"\nRECOMMENDATIONS:")
print("   1. Implement targeted discount programs for lower-satisfaction segments")
print("   2. Focus on upselling to increase purchase amounts and satisfaction")
print("   3. Premium customers already show high satisfaction - maintain service quality")

<a id='presentations'></a>
## 5. Creating Non-Technical Presentations

### Key Principles:
1. **Lead with the Insight** - Start with what you found, not how
2. **Use Visual Storytelling** - Charts beat tables
3. **Avoid Jargon** - Say 'relationship' not 'correlation coefficient'
4. **Quantify Impact** - Show ROI, cost savings, revenue potential
5. **Recommend Actions** - What should they do with this info?

### Example: Executive Summary

In [ ]:
# Create executive summary metrics
total_customers = len(df_clean)
total_revenue = df_clean['purchase_amount_capped'].sum()
avg_purchase = df_clean['purchase_amount_capped'].mean()
avg_satisfaction = df_clean['satisfaction_score'].mean()
top_segment = df_clean.groupby('customer_segment')['purchase_amount_capped'].sum().idxmax()
top_channel = df_clean.groupby('marketing_channel')['purchase_amount_capped'].mean().idxmax()

print("="*60)
print("EXECUTIVE SUMMARY - E-COMMERCE ANALYSIS")
print("="*60)
print(f"\nKEY METRICS")
print(f"   Total Customers: {total_customers:,}")
print(f"   Total Revenue: ${total_revenue:,.2f}")
print(f"   Average Purchase: ${avg_purchase:.2f}")
print(f"   Customer Satisfaction: {avg_satisfaction:.1f}/5.0")

print(f"\nTOP FINDINGS")
print(f"   1. {top_segment} customers drive the highest revenue")
print(f"   2. {top_channel} marketing channel has best ROI")
print(f"   3. Discount usage increases both sales and satisfaction")

print(f"\nRECOMMENDATIONS")
print(f"   1. Invest 40% more in {top_channel} marketing")
print(f"   2. Create loyalty program for {top_segment} customers")
print(f"   3. Expand discount strategies to increase customer lifetime value")
print(f"   4. Focus on Electronics and Clothing categories (highest margins)")

print(f"\nPROJECTED IMPACT")
print(f"   Revenue increase: 15-20% over next quarter")
print(f"   Customer retention: +12%")
print(f"   Customer satisfaction: {avg_satisfaction:.1f} → 4.2+")
print("\n" + "="*60)

### Create Presentation-Ready Visualizations

In [ ]:
# Create a clean, presentation-ready dashboard
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Title
fig.suptitle('E-Commerce Performance Dashboard', fontsize=20, fontweight='bold', y=0.98)

# 1. Revenue by Segment
ax1 = fig.add_subplot(gs[0, 0])
segment_revenue = df_clean.groupby('customer_segment')['purchase_amount_capped'].sum().sort_values(ascending=False)
colors_seg = ['#028090', '#00A896', '#02C39A']
ax1.bar(segment_revenue.index, segment_revenue.values, color=colors_seg, edgecolor='black')
ax1.set_title('Revenue by Segment', fontweight='bold')
ax1.set_ylabel('Revenue ($)')

# 2. Channel Performance
ax2 = fig.add_subplot(gs[0, 1])
channel_avg = df_clean.groupby('marketing_channel')['purchase_amount_capped'].mean().sort_values(ascending=False)
ax2.barh(channel_avg.index, channel_avg.values, color='#028090', edgecolor='black')
ax2.set_title('Avg Purchase by Channel', fontweight='bold')
ax2.set_xlabel('Average Purchase ($)')

# 3. Satisfaction Distribution
ax3 = fig.add_subplot(gs[0, 2])
satisfaction_dist = df_clean['satisfaction_score'].value_counts().sort_index()
ax3.bar(satisfaction_dist.index, satisfaction_dist.values, color='#00A896', edgecolor='black')
ax3.set_title('Satisfaction Distribution', fontweight='bold')
ax3.set_xlabel('Score')
ax3.set_ylabel('Customers')

# 4. Category Performance
ax4 = fig.add_subplot(gs[1, :])
category_stats = df_clean.groupby('product_category').agg({
    'purchase_amount_capped': 'sum',
    'customer_id': 'count'
}).sort_values('purchase_amount_capped', ascending=False)

x = np.arange(len(category_stats))
width = 0.35

ax4_twin = ax4.twinx()
bars1 = ax4.bar(x - width/2, category_stats['purchase_amount_capped'], width,
                label='Revenue', color='#028090', edgecolor='black')
bars2 = ax4_twin.bar(x + width/2, category_stats['customer_id'], width,
                     label='Customers', color='#02C39A', edgecolor='black', alpha=0.7)

ax4.set_xlabel('Product Category')
ax4.set_ylabel('Revenue ($)', color='#028090')
ax4_twin.set_ylabel('Customer Count', color='#02C39A')
ax4.set_title('Product Category Performance', fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(category_stats.index)
ax4.legend(loc='upper left')
ax4_twin.legend(loc='upper right')

# 5. Discount Impact
ax5 = fig.add_subplot(gs[2, 0])
discount_impact = df_clean.groupby('discount_used')['purchase_amount_capped'].mean()
colors_disc = ['#FF6B6B', '#4ECDC4']
ax5.bar(['No Discount', 'With Discount'], discount_impact.values, color=colors_disc, edgecolor='black')
ax5.set_title('Discount Impact on Purchase', fontweight='bold')
ax5.set_ylabel('Avg Purchase ($)')
for i, v in enumerate(discount_impact.values):
    ax5.text(i, v + 2, f'${v:.0f}', ha='center', fontweight='bold')

# 6. Age Distribution
ax6 = fig.add_subplot(gs[2, 1])
ax6.hist(df_clean['age'].dropna(), bins=20, color='#028090', edgecolor='black', alpha=0.7)
ax6.set_title('Customer Age Distribution', fontweight='bold')
ax6.set_xlabel('Age')
ax6.set_ylabel('Count')

# 7. Key Metrics Box
ax7 = fig.add_subplot(gs[2, 2])
ax7.axis('off')
metrics_text = f"""
KEY METRICS

Total Revenue:
${total_revenue:,.0f}

Avg Purchase:
${avg_purchase:.2f}

Satisfaction:
{avg_satisfaction:.1f}/5.0

Customers:
{total_customers:,}
"""
ax7.text(0.1, 0.5, metrics_text, fontsize=12, verticalalignment='center',
         bbox=dict(boxstyle='round', facecolor='#E8F4F8', edgecolor='#028090', linewidth=2),
         fontweight='bold')

plt.savefig('/home/claude/presentation_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("Presentation dashboard created and saved!")

<a id='best-practices'></a>
## 6. Datathon Best Practices

### DO:
- Start with thorough data cleaning
- Spend time understanding the business context
- Create clear, simple visualizations
- Focus on actionable insights
- Practice your presentation
- Document your code and process
- Validate your findings
- Manage your time wisely

### DON'T:
- Skip data quality checks
- Overcomplicate your analysis
- Use jargon in presentations
- Ignore business context
- Wait until last minute to prepare presentation
- Forget to check for outliers
- Make assumptions without validation

### Time Management (8-hour datathon):
- **Hours 1-2:** Data cleaning and initial exploration
- **Hours 3-4:** Deep analysis and visualization
- **Hours 5-6:** Answer business questions
- **Hours 7-8:** Create presentation and practice

## Final Checklist

Before submitting your datathon work:

- [ ] Data is clean (no duplicates, missing values handled, correct types)
- [ ] Outliers identified and addressed appropriately
- [ ] EDA completed with key insights documented
- [ ] Business questions answered with supporting evidence
- [ ] Visualizations are clear and presentation-ready
- [ ] Recommendations are specific and actionable
- [ ] Code is documented and reproducible
- [ ] Presentation tells a compelling story
- [ ] Technical accuracy validated
- [ ] Practice presentation delivered

---

## Additional Resources

### Libraries Documentation:
- **Pandas:** https://pandas.pydata.org/docs/
- **Matplotlib:** https://matplotlib.org/
- **Seaborn:** https://seaborn.pydata.org/
- **Plotly:** https://plotly.com/python/

### Learning Resources:
- Kaggle Learn: https://www.kaggle.com/learn
- DataCamp: https://www.datacamp.com/
- Towards Data Science: https://towardsdatascience.com/

---

**Good luck at your datathon!**